In [1]:
import sys
import numpy as np
import pandas as pd
import csv
import json
from lxml import objectify
import pyarrow
import openpyxl
import xlrd
from bs4 import BeautifulSoup
import sqlalchemy as sqla
import sqlite3
import os
from pathlib import Path

In [2]:
conn = sqlite3.connect("word_for_word.sqlite")
# dataframe to sqlite will put blobs instead of integers if the column has large numbers
# To fix this, register larger integer types
sqlite3.register_adapter(np.int64, int)
sqlite3.register_adapter(np.int32, int)

In [3]:
word_df = pd.DataFrame(columns= ["File", "Words"])

In [4]:
for filename in Path("testdata").iterdir():
    if not filename.suffix == ".txt":
        continue
    with open(filename, 'r') as file:
        data = file.read()
        ser = pd.Series(data.lower().split()).str.replace('[^a-zA-Zà-üÀ-Ü]', '', regex = True)
        word_df = word_df._append([{"File": str(filename), "Words": word} for word in ser])

In [5]:
print(word_df)

                        File     Words
0     testdata/testdata6.txt         p
1     testdata/testdata6.txt   vergili
2     testdata/testdata6.txt   maronis
3     testdata/testdata6.txt  aeneidos
4     testdata/testdata6.txt     liber
..                       ...       ...
683  testdata/testdata5a.txt          
684  testdata/testdata5a.txt    approx
685  testdata/testdata5a.txt          
686  testdata/testdata5a.txt     lines
687  testdata/testdata5a.txt   missing

[163169 rows x 2 columns]


In [6]:
basic_stats = word_df.groupby("File", as_index=True).describe()
print(basic_stats)

                          Words                    
                          count unique    top  freq
File                                               
testdata/testdata0.txt        2      2  hello     1
testdata/testdata1.txt        6      4    zip     2
testdata/testdata2.txt       52     38    the     6
testdata/testdata3.txt    25643   3741    and   717
testdata/testdata4.txt     1335    539    the    78
testdata/testdata5a.txt     688    262    the    65
testdata/testdata6.txt     5035   2904     et   166
testdata/testdata7.txt   130408   7461    the  4842


In [12]:
word_df.groupby("File", as_index=True).describe().to_sql("basic stats", conn, if_exists= 'replace')

8

In [13]:
conn.commit()

In [14]:
cursor = conn.execute("SELECT * FROM 'basic stats'")

In [15]:
rows = cursor.fetchall()

In [16]:
rows

[('testdata/testdata0.txt', 2, 2, 'hello', 1),
 ('testdata/testdata1.txt', 6, 4, 'zip', 2),
 ('testdata/testdata2.txt', 52, 38, 'the', 6),
 ('testdata/testdata3.txt', 25643, 3741, 'and', 717),
 ('testdata/testdata4.txt', 1335, 539, 'the', 78),
 ('testdata/testdata5a.txt', 688, 262, 'the', 65),
 ('testdata/testdata6.txt', 5035, 2904, 'et', 166),
 ('testdata/testdata7.txt', 130408, 7461, 'the', 4842)]

pd.read_sql("SELECT * FROM 'word_for_word", db )